In [1]:
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import tensorflow as tf
import yfinance as yf
import pandas as pd
import numpy as np
import math

In [2]:
company_name= input("Enter the company name(as in listed): ")

Enter the company name(as in listed): AAPL


In [3]:
stock_data = yf.download(tickers=company_name, period='1wk', interval='1m')
stock_data
df1=stock_data['Open']
scaler=MinMaxScaler(feature_range=(0,1))
df1=scaler.fit_transform(np.array(df1).reshape(-1,1))
df1
training_size=int(len(df1)*0.65)
test_size=len(df1)-training_size
train_data,test_data=df1[0:training_size,:],df1[training_size:len(df1),:1]
x_train=[]
y_train=[]
for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
    if i<=61:
        print(x_train)
        print(y_train)
x_train, y_train = np.array(x_train), np.array(y_train)        
x_test=[]
y_test=[]
for i in range(60, len(test_data)):
    x_test.append(train_data[i-60:i, 0])
    y_test.append(train_data[i, 0])
    if i<=61:
        print(x_test)
        print(y_test)
x_test, y_test = np.array(x_test), np.array(y_test)
# reshape input to be [samples, time steps, features] which is required for LSTM
x_train =x_train.reshape(x_train.shape[0],x_train.shape[1] , 1)
x_test = x_test.reshape(x_test.shape[0],x_test.shape[1] , 1)
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(60,1)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64,verbose=1)
train_predict=model.predict(x_train)
test_predict=model.predict(x_test)
##Transformback to original form
train_predict=scaler.inverse_transform(train_predict)
test_predict=scaler.inverse_transform(test_predict)

#Stock Price Forecasting

all_minutes = stock_data['Open']                                    #  df11[-60:]  
all_minutes_scaled = scaler.fit_transform(np.array(all_minutes).reshape(-1,1))
X_test = []
X_test.append(all_minutes_scaled)
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
pred_price = model.predict(X_test)
pred_price = scaler.inverse_transform(pred_price)
print('Predicted Stock Price for the minute: ',pred_price)

[*********************100%***********************]  1 of 1 completed
[array([0.02530304, 0.00937564, 0.01999391, 0.06352655, 0.06901552,
       0.0900706 , 0.10599639, 0.12615879, 0.06033977, 0.08900781,
       0.04972312, 0.        , 0.03166202, 0.0889025 , 0.06564891,
       0.10918155, 0.10918155, 0.10918155, 0.11873702, 0.1357256 ,
       0.12245358, 0.11873702, 0.10918155, 0.10918155, 0.1357256 ,
       0.1357256 , 0.1187273 , 0.10918155, 0.1203296 , 0.1351942 ,
       0.15377537, 0.19199887, 0.18881372, 0.21323379, 0.21597342,
       0.19624683, 0.20534543, 0.21960411, 0.24177384, 0.24137043,
       0.24402581, 0.21089758, 0.22611375, 0.20898746, 0.21217261,
       0.20973109, 0.21854293, 0.23340753, 0.22597604, 0.22491324,
       0.24614816, 0.25901678, 0.27216244, 0.26633325, 0.28437167,
       0.30295283, 0.30613799, 0.31728766, 0.32578033, 0.31888024])]
[0.31197852368130263]
[array([0.02530304, 0.00937564, 0.01999391, 0.06352655, 0.06901552,
       0.0900706 , 0.10599639, 0.1

In [4]:
print('Predicted Stock Price for the minute: ',pred_price)

Predicted Stock Price for the minute:  [[140.33604]]
